# Iris Classification Canary Deployment
This notebook gets you hands-on with Seldon Deploy, demonstrating how you can create canary deployments. 

### Import relevant packages

In [ ]:
!pip install seldon-deploy-sdk

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

import numpy as np
import pandas as pd

import xgboost as xgb

import os
import joblib

### Loading the data
Once you have all of the relevant packages you can now explore the dataset. 

In [ ]:
dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

X = dataset.data
y = dataset.target

Creating a train/test split of the data to ensure you have an unseen subset of the dataset with which to validate your model's performance. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
np.unique(y_train)

Meanwhile, the labels (your y data) have three separate classes- each representing a different species of Iris:

0. Iris setosa
1. Iris virginica
2. Iris versicolor

### Model training
Next you will train two separate machine learning models on your dataset.

In [ ]:
lr = LogisticRegression(max_iter=4000)
lr.fit(X_train, y_train)

print(precision_score(y_test, lr.predict(X_test), average="macro"))
print(recall_score(y_test, lr.predict(X_test), average="macro"))
print(accuracy_score(y_test, lr.predict(X_test)))

You'll notice that your model has perfect evaluation scores, this implies that the model is overfitted on the dataset. For the purposes of this workshop, you're not going to worry about this.

----

You can now train your second machine learning model- an XGBoost classifier. The first step is to convert the dataset into XGBoost's expected DMatrix data format. 

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

Next, you will set the hyperparameters associated with the XGBoost algorithim. This is essentially telling the model how large to be (`max_depth`), how quickly to learn (`eta`), information about the task (`objective`) and  the number of classes (`num_class`). 

Finally, you set the `num_round` which tells XGBoost how many iterations over the dataset to perform. 

In [ ]:
param = {
    'max_depth': 3,
    'eta': 0.3,
    'objective': 'multi:softprob',
    'num_class': 3} 

num_round = 20

You can then train and score the classifier. 

In [ ]:
bst = xgb.train(param, dtrain, num_round)
preds = np.asarray([np.argmax(line) for line in bst.predict(dtest)])
print(precision_score(y_test, preds, average="macro"))
print(recall_score(y_test, preds, average="macro"))
print(accuracy_score(y_test, preds))

Again, you can see that the model has overfit on the dataset. However, you will now save both of the models ready for them to be deployed. 

The Scikit-Learn pre-packaged server expects the saved model artefact to be called `model.joblib`. Meanwhile, the XGBoost pre-packaged server expects your model to be called `model.bst` when it is uploaded to our storage bucket.

In [ ]:
# Saving the logistic regressor
joblib.dump(lr, 'model.joblib')

# Saving the XGBoost classifier
bst.save_model('model.bst')

### Push model artefacts to GCP

You will now push the saved model binaries to a GCP bucket, where they can be picked up by Seldon and deployed onto Kubernetes. 

You will need to create a unique name for your model artefact. It's easiest to just use your own name. Be careful not to use any upper-case letters or other characters like "_". Dashes are fine, so for example: YOUR_NAME = "john-smith"

In [ ]:
YOUR_NAME = "john-smith"

In [ ]:
!gsutil cp model.joblib gs://tom-seldon-examples/deploy-workshop/"{YOUR_NAME}"/lr/model.joblib
!gsutil cp model.bst gs://tom-seldon-examples/deploy-workshop/"{YOUR_NAME}"/xgb/model.bst

### Model Deployment

Deploying the model to a Seldon Deploy trial instance using the `seldon-deploy-sdk`. 

First, setting up the configuration and authentication required to access the cluster. 

In [ ]:
SD_IP = "139.59.203.129"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
username = "admin@seldon.io"
password = "12341234"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate(username, password)
    api_client = ApiClient(config)
    return api_client

### Canary Deployment


Now we have configured the IP correctly as well as setup our authentication function we can describe the deployment we would like to create.

You will need to fill in the model name, deployment name, model location, prepackaged server and traffic split for both models in the canary deployment:

In [ ]:
NAMESPACE = "test"

MODEL_NAME = "lr"
DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/{MODEL_NAME}"
PREPACKAGED_SERVER = "SKLEARN_SERVER"
DEFAULT_TRAFFIC = 60
CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"
CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

CANARY_MODEL_NAME = 'xgb'
CANARY_DEPLOYMENT_NAME = f"{YOUR_NAME}-{CANARY_MODEL_NAME}"
CANARY_MODEL_LOCATION = f"gs://tom-seldon-examples/deploy-workshop/{YOUR_NAME}/{CANARY_MODEL_NAME}"
CANARY_PREPACKAGED_SERVER = "XGBOOST_SERVER"
CANARY_TRAFFIC = 40

In [ ]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": DEFAULT_TRAFFIC,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            },{
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{CANARY_DEPLOYMENT_NAME}-container",
                                    "resources": {}
                                }
                            ]
                        }
                    }
                ],
                "name": "canary",
                "replicas": 1,
                "traffic": CANARY_TRAFFIC,
                "graph": {
                    "implementation": CANARY_PREPACKAGED_SERVER,
                    "modelUri": CANARY_MODEL_LOCATION,
                    "name": f"{CANARY_DEPLOYMENT_NAME}-container"
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.update_seldon_deployment(namespace=NAMESPACE, name=DEPLOYMENT_NAME, mldeployment=mldeployment)

Our canary deployment should now be running as a fully fledged microservice. You can now log into Seldon Deploy and test your deployment:

* URL: http://139.59.203.129/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234

You can now test your models with this request.
```
{
    "data": {
    "names": ["Sepal length","Sepal width","Petal length", "Petal Width"],
    "ndarray": [
        [6.8,  2.8,  4.8,  1.4]
    ]
    }
}
```
  

### Get current deployments



In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.list_seldon_deployments(namespace=NAMESPACE)

### Promote XGBoost

To promote the XGBoost classifier we can adjust our ```mldeployment``` dictionary by replacing our default scikit-learn model with the xgboost model, adjusting the traffic to 100% and removing the canary:

In [ ]:
mldeployment['spec']['predictors'][0] = mldeployment['spec']['predictors'][1]
mldeployment['spec']['predictors'][0]['traffic'] = 100
del mldeployment['spec']['predictors'][1]

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)